# Create Bottleneck

In [14]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import Input, AveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.datasets import cifar10
import pickle
import tensorflow as tf
import keras.backend as K

In [15]:
batch_size = 16

In [16]:
h, w, ch = 299, 299, 3
from keras.applications.inception_v3 import preprocess_input

In [17]:
img_placeholder = tf.placeholder("uint8", (None, 32, 32, 3))
resize_op = tf.image.resize_images(img_placeholder, (h, w), method=0)

In [18]:
def gen(session, data, labels, batch_size):
    def _f():
        start = 0
        end = start + batch_size
        n = data.shape[0]

        while True:
            X_batch = session.run(resize_op, {img_placeholder: data[start:end]})
            X_batch = preprocess_input(X_batch)
            y_batch = labels[start:end]
            start += batch_size
            end += batch_size
            if start >= n:
                start = 0
                end = batch_size

            print(start, end)
            yield (X_batch, y_batch)

    return _f

In [19]:
def create_model():
    input_tensor = Input(shape=(h, w, ch))
    model = InceptionV3(input_tensor=input_tensor, include_top=False)
    x = model.output
    x = AveragePooling2D((8, 8), strides=(8, 8))(x)
    model = Model(model.input, x)
    return model

In [20]:
(X_train, y_train), (_, _) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
with tf.Session() as sess:
        K.set_session(sess)
        K.set_learning_phase(1)

        model = create_model()

        print('Bottleneck training')
        train_gen = gen(sess, X_train, y_train, batch_size)
        bottleneck_features_train = model.predict_generator(train_gen(), X_train.shape[0])
        data = {'features': bottleneck_features_train, 'labels': y_train}
        pickle.dump(data, open('inception_train_output.p', 'wb'))

        print('Bottleneck validation')
        val_gen = gen(sess, X_val, y_val, batch_size)
        bottleneck_features_validation = model.predict_generator(val_gen(), X_val.shape[0])
        data = {'features': bottleneck_features_validation, 'labels': y_val}
        pickle.dump(data, open('inception_validation_output.p', 'wb'))

# Transfer Learning

In [1]:
import pickle
import tensorflow as tf
import numpy as np
from keras.layers import Input, Flatten, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
epochs = 10
batch_size = 128

In [3]:
def load_bottleneck_data(training_file, validation_file):
    """
    Utility function to load bottleneck features.

    Arguments:
        training_file - String
        validation_file - String
    """
    print("Training file", training_file)
    print("Validation file", validation_file)

    with open(training_file, 'rb') as f:
        train_data = pickle.load(f)
    with open(validation_file, 'rb') as f:
        validation_data = pickle.load(f)

    X_train = train_data['features']
    y_train = train_data['labels']
    X_val = validation_data['features']
    y_val = validation_data['labels']

    return X_train, y_train, X_val, y_val

In [22]:
%%bash
ls

03 - mcnulty.ipynb
bottleneck.ipynb
cifar-10-python.tar.gz
data
inception-100.zip
inception_cifar10_100_bottleneck_features_train.p
inception_cifar10_bottleneck_features_validation.p
inception_traffic_100_bottleneck_features_train.p
inception_traffic_bottleneck_features_validation.p
keras_10.h5
keras_30.h5


In [1]:
# TODO: create and use full bottlenecks
X_train, y_train, X_val, y_val = load_bottleneck_data('inception_cifar10_100_bottleneck_features_train.p',
                                                      'inception_cifar10_bottleneck_features_validation.p')

NameError: name 'load_bottleneck_data' is not defined

In [5]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

nb_classes = len(np.unique(y_train))

(1000, 1, 1, 2048) (1000, 1)
(10000, 1, 1, 2048) (10000, 1)


In [6]:
input_shape = X_train.shape[1:]
inp = Input(shape=input_shape)
x = Flatten()(inp)
x = Dense(nb_classes, activation='softmax')(x)
model = Model(inp, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
inception_history = model.fit(X_train, y_train, nb_epoch=epochs, 
                              batch_size=batch_size,
                              validation_data=(X_val, y_val), shuffle=True)
model.save_weights('inception_bottleneck.h5')

Train on 1000 samples, validate on 10000 samples
Epoch 1/10
1000/1000 [==============================] - 1s - loss: 2.2367 - acc: 0.2080 - val_loss: 1.8413 - val_acc: 0.3563
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 1.5103 - acc: 0.5210 - val_loss: 1.4538 - val_acc: 0.5005
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 1.1261 - acc: 0.6720 - val_loss: 1.2599 - val_acc: 0.5699

In [8]:
print(inception_history.history)

{'acc': [0.20800000190734863, 0.52100000190734863, 0.67200000810623173, 0.74700000381469722, 0.79100000619888311, 0.8190000023841858, 0.841999997138977, 0.86900000047683712, 0.8850000047683716, 0.91300000905990597], 'val_acc': [0.35630000000000001, 0.50049999999999994, 0.56989999999999996, 0.60729999999999995, 0.62829999999999997, 0.63849999999999996, 0.64500000000000002, 0.65090000000000003, 0.65510000000000002, 0.65449999999999997], 'val_loss': [1.8413231094360352, 1.4538339244842529, 1.2598726537704468, 1.1618649543762207, 1.1051461982727051, 1.0713191225051879, 1.0496824768066406, 1.0357331386566162, 1.0253588987350464, 1.0191640058517457], 'loss': [2.2366811113357543, 1.510330704689026, 1.1260733051300049, 0.91419462299346921, 0.77806472444534303, 0.67921217727661132, 0.60204377985000612, 0.53972053813934329, 0.48442460250854491, 0.44021475958824158]}
